In [18]:
import os
import numpy as np
import pandas as pd
import plotly.express as px

hierarchy_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/utils/HAD/data/utils/taxonomy_activitynet.csv'
hierarchy = pd.read_csv(hierarchy_path)

video_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/exp/video'
acton_categories = os.listdir(video_path)
parent_categories = hierarchy['parentName'].unique()
parent_categories = np.delete(parent_categories, 0)
acton_categories.extend(parent_categories)

# remove classes that not in the 180 categories
for idx in range(hierarchy.shape[0]):
    tmp_categories = hierarchy.loc[idx, 'nodeName']
    if tmp_categories not in acton_categories:
        hierarchy.drop([idx], axis=0, inplace=True)
        print(f'Categories remove: {tmp_categories}')

Categories remove: Cumbia
Categories remove: Peeling potatoes
Categories remove: Installing carpet
Categories remove: Drinking beer
Categories remove: Layup drill in basketball
Categories remove: Tennis serve with ball bouncing
Categories remove: Changing car wheel
Categories remove: Removing ice from car
Categories remove: Washing face
Categories remove: Shaving legs
Categories remove: Removing curlers
Categories remove: Gargling mouthwash
Categories remove: Curling
Categories remove: Doing a powerbomb
Categories remove: Clipping cat claws
Categories remove: Blowing leaves
Categories remove: Applying sunscreen
Categories remove: Playing ten pins
Categories remove: River tubing
Categories remove: Roof shingle removal


In [ ]:
node_name = hierarchy['nodeName'].to_list()
parent_name = hierarchy['parentName'].to_list()
node_name[0] = 'HAD 180 categories'
node_name[0] = 'HAD 180 categories'

fig = px.treemap(
    names = node_name,
    parents = parent_name
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25), width=1000, height=600,
                  font = dict(family='arial', size=16))
fig.show()

#### Using ete to show hierarchy

In [51]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
from treemaker import TreeMaker
from os.path import join as pjoin

hierarchy_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/utils/HAD/data/utils/taxonomy_activitynet_parentinfo.csv'
hierarchy = pd.read_csv(hierarchy_path)

video_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/exp/video'
result_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/utils/HAD/result'
acton_categories = os.listdir(video_path)


newick_info = '('
parent_names = ['Household Activities', 'Personal Care','Eating and drinking Activities', 
               'Sports, Exercise, and Recreation', 'Socializing, Relaxing, and Leisure']
for super_idx, values in enumerate(hierarchy['parent_3rd'].unique()):
    t = TreeMaker()
    tmp_df = hierarchy[hierarchy.loc[:, 'parent_3rd'] == values]
    tmp_df.reset_index(inplace=True)
    # remove classes that not in the 180 categories
    for idx in range(tmp_df.shape[0]):
        tmp_categories = tmp_df.loc[idx, 'nodeName']
        parent_1st, parent2nd = tmp_df.iloc[idx, 3:5].values
        # if tmp_categories not in acton_categories:
        #     hierarchy.drop([idx], axis=0, inplace=True)
        # write info into treemaker
        t.add(tmp_categories, f'{parent_1st}, {parent2nd}')
    newick_info += t.write().rstrip(';')
    if super_idx != 4:
        newick_info += ','
newick_info += ');'
with open(pjoin(result_path, 'activitynet_newick.txt'), 'w') as f:
    f.write(newick_info)
